# Convert PaddlePaddle Classification Model to ONNX and OpenVINO IR

This tutorial demonstrates step-by-step instructions to convert a PaddlePaddle classification model to OpenVINO IR, and do inference on the converted model.

The model source is https://www.paddlepaddle.org.cn/hubdetail?name=mobilenet_v3_large_imagenet_ssld&en_category=ImageClassification

## Preparation

### Install PaddlePaddle and Upgrade OpenVINO

PaddlePaddle packages are not installed by the default OpenVINO Notebooks installation. Running the cell below checks if they are already installed in the current environment, and installs them if necessary. The Model Optimizer command in this notebook, `mo`, was introduced in OpenVINO 2021.4. If your OpenVINO version is lower, running this cell will upgrade openvino-dev. You can also upgrade all the packages in openvino_env by running `pip install --upgrade -r requirements.txt` in a terminal where you activated the openvino_env environment.

In [ ]:
installed = !pip show paddlepaddle paddlehub paddle2onnx
if "WARNING" in installed.get_nlstr():
    print("Installing PaddlePaddle requirements...")
    !pip install --upgrade "paddlepaddle>=2.1.0" "paddlehub" "paddle2onnx>=0.6"
else:
    import paddle
    import paddle2onnx
    if paddle2onnx.__version__ < "0.6" or paddle.__version__ < "2.1.0":
        print("Upgrading PaddlePaddle requirements...")
        !pip install --upgrade "paddlepaddle>=2.1.0" "paddlehub" "paddle2onnx>=0.6"
print("The PaddlePaddle requirements are installed")

import openvino.inference_engine

if "2021.4" not in openvino.inference_engine.get_version():
    print("Installing OpenVINO 2021.4. This may take a while...")
    print("It is recommended to restart the Jupyter Kernel after installation, "
          "with Kernel->Restart Kernel")
    install_result = %sx python -m pip install --upgrade openvino-dev==2021.4.*
    install_result = [line for line in install_result if "Requirement already " not in line]
    print("\n".join(install_result))
else:
    print("OpenVINO 2021.4 is installed.")

### Imports

In [ ]:
import os
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np
import paddlehub as hub
from IPython.display import Markdown, display
from openvino.inference_engine import IECore
from paddle.static import InputSpec
from PIL import Image
from scipy.special import softmax

### Settings

Set `IMAGE_FILENAME` to the filename of an image to use. Set `MODEL_NAME` to the PaddlePaddle model to download from PaddleHub. `MODEL_NAME` will also be the base name for the converted ONNX and IR models. The notebook is tested with the [mobilenet_v3_large_imagenet_ssld](https://www.paddlepaddle.org.cn/hubdetail?name=mobilenet_v3_large_imagenet_ssld&en_category=ImageClassification) model. Other models may use different preprocessing methods and therefore require some modification to get the same results on the original and converted model.

`hub.config.server` is the URL to the PaddleHub server. You should not need to modify this setting.

In [ ]:
IMAGE_FILENAME = "coco_close.jpg"

MODEL_NAME = "mobilenet_v3_large_imagenet_ssld"
hub.config.server = "https://paddlepaddle.org.cn/paddlehub"

## Show Inference on PaddlePaddle Model

In the next cell, we load and download the model from PaddleHub, read and display an image, do inference on that image, and show the result.

The first time you run this notebook, the PaddlePaddle model is downloaded from PaddleHub. This may take a while.

In [ ]:
classifier = hub.Module(name=MODEL_NAME)
# Load image in BGR format, as specified in model documentation
image = cv2.imread(IMAGE_FILENAME)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
result = classifier.classification(images=[image])
print(result)

`classifier.classification()` takes an image as input and returns the class name of the image. Preprocessing the image and converting the network result to a class name is done behind the scenes. Uncomment the next cell to see the PaddlePaddle source code to load data

In [ ]:
# import mobilenet_v3_large_imagenet_ssld.data_feed
# %load $mobilenet_v3_large_imagenet_ssld.data_feed.__file__

The `data_feed` module shows that images are normalized, resized and cropped and that the BGR image is converted to RGB before propagating through the network. In the next cell, we import the `process_image` function to it to do inference on the OpenVINO IR model with the same method. 

In [ ]:
from mobilenet_v3_large_imagenet_ssld.data_feed import process_image

Show the effect of the `process_image()` function to show the effect of cropping and resizing. Because of the normalization, the colors will look strange, and matplotlib will warn about clipping values. 

In [ ]:
pil_image = Image.open(IMAGE_FILENAME)
processed_image = process_image(pil_image)
print(f"Processed image shape: {processed_image.shape}")
# Processed image is in (C,H,W) format, convert to (H,W,C) to show the image
plt.imshow(np.transpose(processed_image, (1, 2, 0)))

## Convert the Model to OpenVINO IR Format

To Convert the PaddlePaddle model to IR, we first convert the model to ONNX, and then convert the ONNX model to IR.

#### Preparation

PaddlePaddle's MobileNet model contains information about the input shape, mean and scale values that we can use to convert the model. The next cell shows how to get these values.

In [ ]:
input_shape = list(classifier.cpu_predictor.get_input_tensor_shape().values())
print("input shape:", input_shape)
print("mean:", classifier.get_pretrained_images_mean())
print("std:", classifier.get_pretrained_images_std())

#### Convert PaddlePaddle Model to ONNX

We convert the PaddlePaddle Model to ONNX with the `.export_onnx_model()` method. This uses [Paddle2ONNX](https://github.com/PaddlePaddle/paddle2onnx). We convert the model with the input shape found in the previous cell.


In [ ]:
target_height, target_width = next(iter(input_shape))[2:]
x_spec = InputSpec([1, 3, target_height, target_width], "float32", "x")
print(
    "Exporting PaddlePaddle model to ONNX with target_height"
    f"{target_height} and target_width {target_width}"
)
classifier.export_onnx_model(".", input_spec=[x_spec], opset_version=11)

#### Convert ONNX model to OpenVINO IR Format

Call the OpenVINO Model Optimizer tool to convert the PaddlePaddle model to OpenVINO IR, with FP16 precision. The models are saved to the current directory. We can add the mean values to the model with `--mean_values` and scale the output with the standard deviation with `--scale_values`. With these options, it is not necessary to normalize input data before propagating it through the network. However, to get the exact same output as the PaddlePaddle model, it is necessary to preprocess in the image in the same way. For this tutorial, we therefore do not add the mean and scale values to the model, and we use the `process_image` function, as described in the previous section, to ensure that both the IR and the PaddlePaddle model use the same preprocessing methods. We do show how to get the mean and scale values of the PaddleGAN model, so you can add them to the Model Optimizer command if you want. See the [PyTorch/ONNX to OpenVINO](../102-pytorch-onnx-to-openvino/102-pytorch-onnx-to-openvino.ipynb) notebook for a notebook where these options are used.

See the [Model Optimizer Developer Guide](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) for more information about Model Optimizer, including a description of the command line options. 

In the next cell, we first construct the command for Model Optimizer, and then execute this command in the notebook by prepending the command with a `!`. Model Optimization was succesful if the last lines of the output include `[ SUCCESS ] Generated IR version 10 model.

In [ ]:
model_xml = f"{MODEL_NAME}.xml"
if not os.path.exists(model_xml):
    mo_command = f'mo --input_model {MODEL_NAME}.onnx --input_shape "[1,3,{target_height},{target_width}]"'
    display(Markdown(f"Model Optimizer command to convert the ONNX model to IR: `{mo_command}`"))
    display(Markdown("_Converting model to IR. This may take a few minutes..._"))
    ! $mo_command
else:
    print(f"{model_xml} already exists.")

## Show Inference on OpenVINO model

Load the IR model, get model information, load the image, do inference, convert the inference to a meaningful result, and show the output. See the [Inference Engine API Notebook](../002-openvino-api/002-openvino-api.ipynb) for more information.

In [ ]:
# Load Inference Engine and IR model
ie = IECore()
net = ie.read_network(f"{MODEL_NAME}.xml")
exec_net = ie.load_network(net, "CPU")

# Get model input and outputs
input_layer = next(iter(net.input_info))
output_layer = next(iter(net.outputs))

# Read, show, and preprocess input image
# See the "Show Inference on PaddlePaddle Model" section for source of process_image
image = Image.open(IMAGE_FILENAME)
plt.imshow(image)
input_image = process_image(image)

# Do inference
ie_result = exec_net.infer({input_layer: input_image})[output_layer][0]

# Convert inference result to a class name, using the same labels as the PaddlePaddle classifier
result_index = np.argmax(ie_result)
class_name = classifier.label_list[result_index]

# Compute the softmax score of the result
softmax_result = softmax(ie_result)[result_index]
print(class_name, softmax_result)

## Timing and Comparison

Measure the time it takes to do inference on fifty images and compare the result. The timing information gives an indication of performance. For a fair comparison, we include the time it takes to preprocess the image. For more accurate benchmarking, use the [OpenVINO benchmark tool](https://github.com/openvinotoolkit/openvino/tree/master/inference-engine/tools/benchmark_tool). Note that many optimizations are possible to improve the performance.

In [ ]:
num_images = 50
# PaddlePaddle's classification method expects a BGR numpy array
image = cv2.imread(IMAGE_FILENAME)
# The process_image function expects a PIL image
pil_image = Image.open(IMAGE_FILENAME)

In [ ]:
start = time.perf_counter()
for _ in range(num_images):
    paddle_result = classifier.classification(images=[image])
end = time.perf_counter()
time_ir = end - start
print(
    f"PaddlePaddle model on CPU: {time_ir/num_images:.4f} "
    f"seconds per image, FPS: {num_images/time_ir:.2f}"
)
print(f"PaddlePaddle result: {paddle_result}")

In [ ]:
start = time.perf_counter()
for _ in range(num_images):
    input_image = process_image(pil_image)
    ie_result = exec_net.infer(inputs={input_layer: input_image})[output_layer][0]
end = time.perf_counter()
time_ir = end - start


# Convert the inference result to a class name, using the same labels as the PaddlePaddle classifier
result_index = np.argmax(ie_result)
class_name = classifier.label_list[np.argmax(ie_result)]
softmax_result = softmax(ie_result)[result_index]

print(
    f"IR model in Inference Engine/CPU: {time_ir/num_images:.4f} "
    f"seconds per image, FPS: {num_images/time_ir:.2f}"
)
print(f"OpenVINO result: {[class_name, softmax_result]}")